<a href="https://colab.research.google.com/github/yukuproj/made2/blob/main/hw2_what_where_when.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Что? Где? Когда? Рейтинги homework 2




In [2]:
import numpy as np
import pandas as pd
import pickle
import re
import scipy.stats as stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from scipy import sparse as sp
from scipy.special import expit


In [3]:
ls

drive/  sample_data/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Чтение

In [5]:
players_file = "/content/drive/MyDrive/made2_ml_hw2/players.pkl"
results_file = "/content/drive/MyDrive/made2_ml_hw2/results.pkl"
tournaments_file = ("/content/drive/MyDrive/made2_ml_hw2/tournaments.pkl")

with open(players_file, 'rb') as fin:
     players_data = pickle.load(fin)

with open(tournaments_file, 'rb') as fin:
     tournaments_data = pickle.load(fin)

with open(results_file, 'rb') as fin:
     results_data = pickle.load(fin)     

Filtering data

In [6]:
def isOk(tr):
    rez1 = tr.get('mask') and tr.get('teamMembers')    
    rez2 = len(tr.get('teamMembers')) > 0    
    rez3 = all(players_data.get(tm['player']['id']) for tm in tr.get('teamMembers'))
    if not(rez1 and rez2 and rez3):  ##  нет состава команд или детализации по вопросам
      return False
    rez4 = not re.findall('[^01]', tr.get('mask'))   ## нет четких ответов
    return rez4

In [7]:
results_filtered = {}
tournaments_filtered = {}

for t_id, team_results in results_data.items():
    team_results_filtered = [tr for tr in team_results if isOk(tr)]
    if (len(team_results_filtered) > 0):
        results_filtered[t_id] = team_results_filtered
        tournaments_filtered[t_id] = tournaments_data[t_id]

In [8]:
tournaments_train = {k: v for k, v in tournaments_filtered.items() if v['dateStart'][:4] == '2019'}  ## тестовая выборка -турниры за 2020 год
tournaments_test  = {k: v for k, v in tournaments_filtered.items() if v['dateStart'][:4] == '2020'}  ## обучающая выборка - турниры за 2019 год

In [9]:
print(f'Размер обучающей выборки: {len(tournaments_train)}')
print(f'Размер тестовой выборки:  {len(tournaments_test)}')

Размер обучающей выборки: 616
Размер тестовой выборки:  160


## 2. Baseline-модель

Будем предполагать что игрок отвечает точно также как и его команда. Т.е. если команда на вопрос ответила, то это значит, что и каждый игрок внутри команды тоже способен был самостоятельно ответить на вопрос. И обратно, более естественное требование, если команда не ответила, то не смог бы ответить и каждый игрок. 

Будем считать что модель устроена как логистическая регрессия, в которой в качестве аргументов являются метки игрока и метка вопроса. Неизвестными коэффициентами в этой регрессии будут являться "уровень" игрока и "сложность вопроса". При этом обучаться будем на 0 или 1 в зависимости от того, ответил ли игрок на вопрос или нет.

Для определения переменных модели закодируем игрока и вопрос в с помощью OneHotEncoding представления. Входной вектор для обучения тогда - это вектор из нулей и единиц, где единицы соответствуют конкретному игроку и конкретному вопросу на который он отвечает, а ответ - 0 или 1 в зависимости от того, ответил ли игрок на вопрос. 




In [10]:

train_table = []
max_question_id = 0
for tour_id, _ in tournaments_train.items():
    for team_result in results_filtered[tour_id]:
        team_id = team_result['team']['id']
        mask = np.array([np.int32(answer) for answer in team_result['mask']])
        players = team_result['teamMembers']
        questions = np.arange(max_question_id, max_question_id + len(mask))
        
        for player in players:
            player_id = player['player']['id']
            for i in range(len(mask)):
                train_table.append([tour_id, team_id, player_id, questions[i], mask[i]])
    max_question_id += len(mask)    
        
train_df = pd.DataFrame(train_table, columns = ['tour_id', 'team_id', 'player_id', 'question_id', 'answer'])      

In [11]:
ohe = OneHotEncoder(handle_unknown='ignore')
X_train = ohe.fit_transform(train_df[['player_id', 'question_id']])
y_train = train_df['answer']

In [12]:
lr = LogisticRegression(n_jobs = -1)
lr.fit(X_train, y_train)

LogisticRegression(n_jobs=-1)

In [13]:
active_players = np.unique(train_df['player_id'])
rating = pd.DataFrame({'id': active_players, 'coef': lr.coef_[0][:len(active_players)], 'name': [(players_data[i]['name'] + ' ' + players_data[i]['surname']) for i in active_players]})

In [14]:
rating.sort_values(by='coef', ascending=False).head(50)

,id,coef,name
3803,27403,3.589675,Максим Руссо
593,4270,3.537738,Александра Брутер
5106,37047,3.384659,Мария Юнгер
3991,28751,3.311466,Иван Семушин
5289,38196,3.247600,Артём Митрофанов
513,3671,3.213598,Алексей Богословский
3874,27822,3.200231,Михаил Савченков
4176,30152,3.135241,Артём Сорожкин
583,4226,3.118063,Сусанна Бровер
6633,56647,3.102888,Наталья Горелова


## 3. Качество рейтинговой системы

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров.

Чтобы оценить качество, возьмем тестовую выборку и для каждого вопроса будем считать, что чтобы команда ответила на вопрос надо чтобы хотя бы один игрок ответил на этот вопрос.


In [15]:
test_table = []

for tournament_id, tournament in tournaments_test.items():
    for team_result in results_filtered[tournament_id]:
        team_id = team_result['team']['id']
        mask = np.array([np.int32(answer) for answer in team_result['mask']])
        players = team_result['teamMembers']
        
        for player in players:
            player_id = player['player']['id']
            if player_id not in active_players:
                continue
            test_table.append([tournament_id, team_id, player_id, -1, sum(mask), len(mask)])  
        
test_df = pd.DataFrame(test_table, columns = ['tournament_id', 'team_id', 'player_id', 'question_id', 'correct_answers', 'total_answers'])      

In [16]:
X_test = test_df[['player_id', 'question_id']]
X_test = ohe.transform(X_test)
predictions = lr.predict_proba(X_test)[:, 1]

In [17]:
def compute_scores(data, preds):
    data['pred'] = preds
    data['score'] = data.groupby(['tournament_id', 'team_id'])['pred'].transform(lambda x: 1 - np.prod(1 - x))
    rating = data[['tournament_id', 'team_id', 'correct_answers', 'score']].drop_duplicates().reset_index(drop=True)
    
    # Считаем реальный рейтинг команд
    rating = rating.sort_values(by=['tournament_id', 'correct_answers'], ascending=False)
    rating['real_rank'] = rating.groupby('tournament_id')['correct_answers'].transform(lambda x: np.arange(1, len(x) + 1))
    
    # Считаем предсказанный рейтинг
    rating = rating.sort_values(by=['tournament_id', 'score'], ascending=False)
    rating['pred_rank'] = rating.groupby('tournament_id')['score'].transform(lambda x: np.arange(1, len(x) + 1))

    rating = rating.astype(np.int32)
    
    print(f"Корреляция Спирмана: {rating.groupby('tournament_id').apply(lambda x: stats.spearmanr(x['real_rank'], x['pred_rank']).correlation).mean()}")
    print(f"Корреляция Кендалла: {rating.groupby('tournament_id').apply(lambda x: stats.kendalltau(x['real_rank'], x['pred_rank']).correlation).mean()}")

In [18]:
compute_scores(test_df, predictions)

Корреляция Спирмана: 0.7672515187411473
Корреляция Кендалла: 0.5987298826719607


## 4. EM алгоритм

Будем считать, что теперь все игроки в команде могли отвечать по-разному. Даже если вся команда ответила на вопрос, то будем считать, что это не означает, что каждый ответил на вопрос. Если же команда не ответила на вопрос, то как и ранее будем считать, что каждый из игроков на вопрос не ответил.

Формально можно считать, что для каждого вопроса j и каждого игрока i есть некоторая случайная ненаблюдаемая! величина z_i_j, которая принимает два значения 0 или 1 в зависимости от того, смог бы игрок i самостоятельно ответить на вопрос j.

Сами значения z_i_j мы непосредственно не наблюдаем, поскольку они замешаны с ответами команды и результаты игрока оказываются зашумлены ответами команды. Однако, ответы команды и ответы игрока коррелированы, что даст возможность через ответы команды получить некоторую информацию об гипотетических ответах самого игрока.

Подсчитаем условную вероятность того, что игрок ответит, при условии что его команда не ответила (очевидно эта вероятность 0) и при условии, что его команда ответила: 

 

Введем новые обозначения

Пусть 
 $P (z_{i,j}=1| x) $ - условная вероятность того, что игрок ответит на вопрос, при условии что его команда ответит на вопрос либо верно (x=1), либо неверно (x=0).

Как уже отмечалось,  $P (z_{i,j}=1| x=0) = 0$ - если команда не ответила на вопрос, то и каждый игрок не стал бы утаивать правильный ответ и значит сам бы не ответил на вопрос.

Если же команда ответила на вопрос, условная вероятность сложнее (по формуле полной вероятности):
 
 $P (z_{i,j}=1| x=1) = \frac{P(z_{i,j}=1)}{P(x=0)} = \frac{P(z_{i,j}=1)} {1-(1-P(z_{i_{1},j }=1))...(1-P(z_{i_{n},j }=1))}$

 где $z_{i_{k},j }$ - аналогичные события что игрок $i_{k}$ из той же команды что и игрок i ответит верно на вопрос j. Произведение в знаменателе по всем игрокам из команды.

 Алгоритм выглядит таким образом:

На M шаге подбираем коэффициенты в логистической регрессии как и ранее в baseline, только в качестве целей используем не 0 или 1 как ранее, а найденные условные вероятности $P (z_{i,j}=1| x)$.

На E шаге обновляем условные вероятности

 $P (z_{i,j}=1| x=0) = 0$
 
 $P (z_{i,j}=1| x=1) = \frac{P(z_{i,j}=1)}{P(x=0)} = \frac{P(z_{i,j}=1)} {1-(1-P(z_{i_{1},j }=1))...(1-P(z_{i_{n},j }=1))}$





In [19]:
def log_loss(y_true, y_pred):
    return - np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

class EMClassifier:
    
    def __init__(self, w=None, lr=15, n_iter=10, batch_size=5000, verbose=1):
        self.w = w
        self.lr = lr
        self.n_iter = n_iter
        self.batch_size = batch_size
        self.verbose = 1
        
    def _add_intercept(self, X):
        return sp.hstack((np.ones((X.shape[0], 1)), X), format='csr')
    
        
    def _E_step(self, data, preds):
        team_prob = pd.DataFrame({'team_id': data['team_id'], 'question_id': data['question_id'], 'team_prob': 1 - preds}).groupby(['team_id', 'question_id']).agg({'team_prob': 'prod'}).reset_index()

        team_prob['team_prob'] = 1 - team_prob['team_prob']
        team_prob = data[['team_id', 'question_id']].merge(team_prob)
        y = np.clip(preds / team_prob['team_prob'], 0, 1).values
        y[data['answer'] == 0] = 0
        return y
        
    def _M_step(self, X, y):
        min_loss = np.inf
        indices = np.arange(X.shape[0])
        for _ in range(100):
            indices = np.random.permutation(indices)
            for batch_idx in np.array_split(indices, len(indices) // self.batch_size):
                x_batch, y_batch = X[batch_idx], y[batch_idx]
                grad = x_batch.T.dot(self.predict(x_batch) - y_batch) / len(y_batch)
                self.w -= self.lr * grad
                
            cur_loss = log_loss(y, self.predict(X))
            if min_loss - cur_loss < 1e-6:
                break
                
            min_loss = cur_loss
                
    def fit(self, X_tr, train_data, X_te=None, test_data=None):
        X_tr = self._add_intercept(X_tr)
        for iter_ in range(self.n_iter): 
            preds = self.predict(X_tr)
            y = self._E_step(train_data, preds)
            self._M_step(X_tr, y)
            if self.verbose is not None and X_te is not None and test_data is not None and iter_ % self.verbose == 0:
                compute_scores(test_data, self.predict(X_te))
                         
    def predict(self, X):
        if len(self.w) != X.shape[1]:
            X = self._add_intercept(X)
        return expit(X.dot(self.w))

In [20]:
w_init = np.hstack([lr.intercept_, lr.coef_[0]]) # инициализация весов из baseline
em_classifier = EMClassifier(w_init)

em_classifier.fit(X_train, train_df, X_test, test_df)

Корреляция Спирмана: 0.7716524230787055
Корреляция Кендалла: 0.603905714706942
Корреляция Спирмана: 0.7755026286026792
Корреляция Кендалла: 0.6083365789573657
Корреляция Спирмана: 0.775811579841464
Корреляция Кендалла: 0.6082407043712538
Корреляция Спирмана: 0.7758907759478948
Корреляция Кендалла: 0.6085188808788724
Корреляция Спирмана: 0.7757402794919488
Корреляция Кендалла: 0.6085573799501848
Корреляция Спирмана: 0.7746745799749475
Корреляция Кендалла: 0.6073159145151498
Корреляция Спирмана: 0.7753439962718177
Корреляция Кендалла: 0.6081549668638893
Корреляция Спирмана: 0.7770330492665783
Корреляция Кендалла: 0.6092810722004044
Корреляция Спирмана: 0.7773268284663192
Корреляция Кендалла: 0.6094622991336516
Корреляция Спирмана: 0.777626231356631
Корреляция Кендалла: 0.6100120464562828


Модель немного лучше чем baseline

## 5. Рейтинг лист турниров



Считаем среднюю сложность вопросов в турнире (среднее коэффициентов в модели).

In [23]:
active_questions = np.unique(train_df['question_id'])
q_rating = dict(zip(active_questions, em_classifier.w[-len(active_questions):]))

train_df['difficulty'] = train_df['question_id'].map(q_rating)
train_df['tournament_name'] = train_df['tour_id'].map({v['id']: v['name']for k, v in tournaments_data.items()})

tournaments_rating = train_df[['tournament_name', 'question_id', 'difficulty']].drop_duplicates()
tournaments_rating = tournaments_rating.groupby('tournament_name')['difficulty'].mean().sort_values().reset_index()

In [25]:
tournaments_rating.head(10)

,tournament_name,difficulty
0,Чемпионат Санкт-Петербурга. Первая лига,-3.431736
1,Угрюмый Ёрш,-2.031730
2,Первенство правого полушария,-1.867021
3,Кубок городов,-1.644960
4,Воображаемый музей,-1.546082
5,Записки охотника,-1.468422
6,Чемпионат России,-1.468335
7,Ускользающая сова,-1.427819
8,All Cats Are Beautiful,-1.413481
9,VERSUS: Коробейников vs. Матвеев,-1.383124
